# 자율주행 선박을 위한 AI 알고리즘 구현 실습

<br /> <span style="font-family: Open Sans; color: red; font-weight: 800; font-size: 20px;">코드 실행 안내</span>
- 코드 실행 셀의 좌측에 있는 In[  ] 의 괄호 안에 * 가 표시된경우, 코드가 실행 중이라는 의미입니다.
-  In[  ] 의 괄호 안에 숫자가 표시될때까지 기다려 주세요.

***
# 0. MODI 모듈 준비 & 패키지 설치
#### 이번 컨텐츠에는 아래와 같은 모듈이 필요합니다.
- 입력 모듈
    - Gyro 모듈
    - Button 모듈
- 출력 모듈
    - Motor 모듈

- 설정 모듈
    - Network 모듈

### 모듈 셋업

1. 모터 모듈과 네트워크 모듈을 연결하고 자동차 형태로 조립합니다.
2. 자이로 모듈과 버튼 모듈을 붙여 리모컨을 만듭니다.
3. 긴 CAN 케이블을 사용하여 리모컨과 자동차의 네트워크 모듈을 연결합니다.
4. 네트워크 모듈과 PC를 USB 케이블을 사용하여 연결합니다.

### pymodi와 tensorflow 설치하기
pymodi와 tensorflow 모듈이 설치되어 있지 않은 경우 아래 코드를 실행하여 설치합니다.
코드 블럭 좌측의 In[ ] 괄호 안의 * 가 숫자로 바뀐 후에 다음 코드를 실행하세요. 
- pymodi: MODI를 컨트롤하기 위한 API
- tensorflow: 머신러닝을 위한 라이브러리

In [ ]:
!python -m pip install pymodi --user
!python -m pip install tensorflow --user

***
# 1. 데이터 수집
## 학습에 사용할 데이터를 수집합니다. 

1-1. gathering_data 모듈과 Pymodi 패키지를 import합니다.

In [ ]:
from gathering_data import DataGathering
import modi

1-2. 모디 객체와 DataGathering 클래스 객체를 생성합니다.

MODI 객체를 생성합니다.
Gyro, Button, Motor 모듈을 각각 gyro, btn, mot 변수에 할당합니다.

In [ ]:
bundle = modi.MODI()

gyro = bundle.gyros[0]
btn = bundle.buttons[0]
mot = bundle.motors[0]

dg = DataGathering()

1-3. go, back, left, right 4개 동작에 대한 데이터를 수집합니다. 파일 이름(go, back, left, right)을 입력하고, gathering_motion 메서드를 실행하여 데이터를 수집합니다. 

이 때 데이터 파일에는 자이로 모듈이 측정한 각 동작의 가속도와 회전 각도 값이 저장됩니다.

In [ ]:
filename = 'left' # 파일은 /data/go.csv 에 저장됩니다.
dg.gathering_motion(btn, gyro, filename)

1-4. 현재까지 수집된 데이터의 개수를 확인합니다.

In [ ]:
dg.check_data()

# 2. 모델 학습

## 수집한 데이터를 통해 학습 모델을 생성합니다.

2-1. gesture_detection 모듈을 import 합니다.

In [ ]:
from gesture_detection import DetectGesture

2-2. DetectGesture 클래스 객체를 생성합니다.

In [ ]:
dg = DetectGesture()

2-3. training_model 메서드를 실행하여 수집한 데이터를 사용하여 학습하고, 모델을 생성합니다.

In [ ]:
dg.training_model()

# 3. 학습된 모델을 활용하여 자율주행 콘텐츠 제작

## 생성된 모델을 활용하여 자율주행 콘텐츠를 제작합니다.

3-1. gesture_detection 모듈과 Pymodi 및 필수 패키지들을 import 합니다.

In [ ]:
from gesture_detection import DetectGesture
from IPython.display import clear_output
import modi
import time

3-2. 모디 객체와 DetectGesture 클래스 객체를 생성합니다.

MODI 객체를 생성합니다.
Gyro, Button, Motor 모듈을 각각 gyro, btn, mot 변수에 할당합니다.

In [ ]:
bundle = modi.MODI()

gyro = bundle.gyros[0]
btn = bundle.buttons[0]
mot = bundle.motors[0]

In [ ]:
dg = DetectGesture()

3-3. 반복문 안에 있는 사용자 코드 영역을 수정하여 모터 모듈이 움직이도록 코드를 작성합니다.

코드를 실행하면 자이로 모듈이 수집한 데이터에 맞게 MODI 자동차가 움직이게 됩니다.

In [ ]:
while True:
        time.sleep(0.1)
        print("버튼을 더블클릭하면 출발합니다")
        clear_output(wait=True)
        if btn.double_clicked:
            print("출발!")
            mot.speed = 35, -35
            time.sleep(0.1)
            while True:
                print("버튼을 눌러 데이터를 수집해보세요. 멈추려면 버튼을 더블클릭하세요.", end='\r')
                if btn.clicked:
                    pred = dg.predict(gyro, btn) #pred 변수에, 예측값을 받아와 저장합니다.
                    time.sleep(0.1)
                    # 사용자 코드 영역
                    # =======================================================
                    if pred == 'back':
                        mot.speed = -30,30
                        time.sleep(0.1)
                        print('car : back!')
                    elif pred == 'go':
                        mot.speed = 30, -30
                        time.sleep(0.1)
                        print('car : go!')
                    elif pred == 'left':
                        mot.speed = 30, 30
                        time.sleep(0.1)
                        print('car : left!')
                    elif pred == 'right':
                        mot.speed = -30, -30
                        time.sleep(0.1)
                        print('car : right!')
                    #=======================================================

                time.sleep(0.1)
                if btn.double_clicked:
                    print("stop!")
                    mot.speed = 0, 0
                    time.sleep(0.1)
                    break